In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-tauwz9ik
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-tauwz9ik
  Resolved https://github.com/huggingface/transformers to commit 093848d3ccf3884caf048718b6bae833da0edb94
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7817578 sha256=0b036ec0293edf5ec5706555ce88f31fba7cfc866ad6d0c82002b7baa90d39ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-vwhml_uc/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successful

In [ ]:
!nvidia-smi

Sun Oct 22 09:53:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Mistral-7B-Instruct**

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.set_default_device("cuda")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                          torch_dtype = "auto")


model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             load_in_8bit=True,
                                             torch_dtype = "auto")



OutOfMemoryError: ignored

**Prompt Format**


In [ ]:
'''
<s>[INST]<user_prompt>[/INST]

<assistant_prompt></s>

[INST]<>[/INST]
'''

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve newlines
  lines = text.split('\n') #split text-to-lines based on newlines

  #wrap each line individually
  wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

  #join the wrapped lines back together using newline
  wrapped_text = "\n".join(wrapped_lines)

  return wrapped_text

In [ ]:
def generate(input_text, system_prompt="", max_length=512):
  prompt = f"""<s>[INST]{input_text}[/INST]"""
  inputs = tokenizer(input_text, return_tensors="pt", add_special_token=False)
  model_inputs = encodeds.to(device)
  model.to(device)
  outputs = model.generate(**inputs,
                           max_length= max_length,
                           temprature=0.1,
                           do_sample= True)
  text  = tokenizer.batch_decode(outputs)[0]
  wrapped_text = wrap_text(text)
  print(wrapped_text)



**CodeGen**

In [ ]:
generate('''python
         def print_prime(n):
          """
          print all prime between 1 and n
          """''', system_prompt="You are a genius python coder")


**Instruction Answering**

In [ ]:
%%time
generate("Write a detailed analogy  between mathematics and a lighthouse",
         max_length = 128)

In [ ]:
%%time
generate("Write  a detailed analogy between mathematics and music",
         max_length = 256)

In [ ]:
%%time
generate("what is the difference between LLama, Vicuna and Alpaca?",
         max_length = 512)

In [ ]:
%%time
generate("Write a short email Arun Kumar giving reasons why his experinece in nlp, llm and machine learningis great fit the data scientist job",
         max_length = 512)

In [ ]:
%%time
generate("What is the capital of India??",
         max_length = 256)

**GSM8K dataset questions**

In [ ]:
generate("  ",
         max_length =256)